## Fase 4: Limpieza de datos

Tenemos más de 200 columnas de las que sólo queremos las concernientes a sus datos de programación

Nos vamos a quedar con las siguientes columnas

* languages: la correspondientes a cada uno de los lenguajes soportados en codewars
* stats sociales: allies, clan, comments, followers, following, github, etc. 
* stats estadísticos: honor, last seen, leaderboard position, etc. 


In [86]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display
import time

from funciones_scraping import get_languages

pd.options.display.max_columns = None

In [87]:
df = pd.read_csv('../output/df-codewars-iter3.csv', index_col=0)

In [88]:
df.shape

(11501, 238)

In [89]:
df.head()

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,github,skills,linkedin,algebra,algorithms,avg. rank,avg. satisfaction rating,created,fundamentals,logic,mathematics,numbers,total collected,total completions,total stars,arithmetic,arrays,data types,computability theory,recursion,theoretical computer science,games,puzzles,utilities,3 kyu,4 kyu,5 kyu,6 kyu,7 kyu,8 kyu,authored kata,authored translations,best practice,best practice solutions,clever,clever solutions,completed kata,completed on 1st attempt,contributed kata,current streak,date,ended on,first completed,kata approvals,kata attempts,last completed,most number of days,basic language features,classes,control flow,functions,modules,programming paradigms,binary,data,strings,2 kyu,validation,exception handling,security,data structures,linked lists,lists,geometry,functional programming,computational science,state machines,expressions,ascii,character encodings,dates/time,formats,decoding,encoding,prototypes,graphs,parsing,advanced language features,regular expressions,1 kyu,loops,objects,object-oriented programming,dynamic programming,data conversion,immutability,mutability,statistics,sorting,declarative programming,interpreters,optimization,performance,integers,cryptography,encryption,refactoring,bugs,io,streams,design patterns,memoization,permutations,trees,bash,networks,search,iterators,conditional statements,asynchronous,design principles,promises,singleton,esoteric languages,formatting,higher-order functions,filtering,best practices,machine learning,decryption,babel,es2015,physics,game boards,variables,big integers,sets,frameworks,react,lambdas,hashes,methods,databases,information systems,logic programming,applied computer science,angular,observers,hacking holidays,vectors,binary search trees,rules,nodejs,queues,ciphers,sequences,memory,metaprogramming,reflection,chars,django,tables,arguments,dictionary,case/switch statements,map/reduce,bits,inheritance,polymorphism,scopes,event handling,jsx,testing,graphics,prototype-based programming,unicode,recursion theory,json,interview questions,ranking,weak typing,maps,reporting,operators,closures,sparse arrays,properties,booleans,bitwise operators,decimals
0,1.0,0.0,0.0,0.0,0.0,Hagerty,0.0,0.0,0.0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,JavaScript (5 kyu),560,96th,0.0,581.0,0.0,0.0,0,Aug 2018,"#19,926",0.0,Oct 2014,C#,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45,2.0,0 (0 approved),0.0,brkiesel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,374.0,0.0,0.0,0.0,0.0,FreeCodeCamp,0.0,0.0,1.0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,377.0,386.0,0.0,0.0,0.0,0.0,JavaScript (3 kyu),"1,984",99th,14.0,4449.0,0.0,0.0,0,Sep 2018,"#2,492",0.0,Sep 2015,Python,Emanuele,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13

### 4.0 Drop columnas con muchos nulos

Eliminamos las columnas con más de la mitad de sus registros a NaN (no incluyendo github ni linkedind)

In [90]:
# Vemos los nulos. 
redes_sociales = ['linkedin', 'github']
columnas_nulas = [col for col in df.columns if df[col].isna().sum()>len(df)//2 and col not in redes_sociales]
len(columnas_nulas)

178

In [91]:
df.head()

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,github,skills,linkedin,algebra,algorithms,avg. rank,avg. satisfaction rating,created,fundamentals,logic,mathematics,numbers,total collected,total completions,total stars,arithmetic,arrays,data types,computability theory,recursion,theoretical computer science,games,puzzles,utilities,3 kyu,4 kyu,5 kyu,6 kyu,7 kyu,8 kyu,authored kata,authored translations,best practice,best practice solutions,clever,clever solutions,completed kata,completed on 1st attempt,contributed kata,current streak,date,ended on,first completed,kata approvals,kata attempts,last completed,most number of days,basic language features,classes,control flow,functions,modules,programming paradigms,binary,data,strings,2 kyu,validation,exception handling,security,data structures,linked lists,lists,geometry,functional programming,computational science,state machines,expressions,ascii,character encodings,dates/time,formats,decoding,encoding,prototypes,graphs,parsing,advanced language features,regular expressions,1 kyu,loops,objects,object-oriented programming,dynamic programming,data conversion,immutability,mutability,statistics,sorting,declarative programming,interpreters,optimization,performance,integers,cryptography,encryption,refactoring,bugs,io,streams,design patterns,memoization,permutations,trees,bash,networks,search,iterators,conditional statements,asynchronous,design principles,promises,singleton,esoteric languages,formatting,higher-order functions,filtering,best practices,machine learning,decryption,babel,es2015,physics,game boards,variables,big integers,sets,frameworks,react,lambdas,hashes,methods,databases,information systems,logic programming,applied computer science,angular,observers,hacking holidays,vectors,binary search trees,rules,nodejs,queues,ciphers,sequences,memory,metaprogramming,reflection,chars,django,tables,arguments,dictionary,case/switch statements,map/reduce,bits,inheritance,polymorphism,scopes,event handling,jsx,testing,graphics,prototype-based programming,unicode,recursion theory,json,interview questions,ranking,weak typing,maps,reporting,operators,closures,sparse arrays,properties,booleans,bitwise operators,decimals
0,1.0,0.0,0.0,0.0,0.0,Hagerty,0.0,0.0,0.0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,JavaScript (5 kyu),560,96th,0.0,581.0,0.0,0.0,0,Aug 2018,"#19,926",0.0,Oct 2014,C#,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45,2.0,0 (0 approved),0.0,brkiesel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,374.0,0.0,0.0,0.0,0.0,FreeCodeCamp,0.0,0.0,1.0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,377.0,386.0,0.0,0.0,0.0,0.0,JavaScript (3 kyu),"1,984",99th,14.0,4449.0,0.0,0.0,0,Sep 2018,"#2,492",0.0,Sep 2015,Python,Emanuele,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13

In [92]:
# hacemos drop de las columnas que tengan más del 50% a nulos

df.drop(columns=columnas_nulas, inplace=True)
df.columns

Index(['allies', 'bf (beta)', 'c', 'c#', 'c++', 'clan', 'clojure',
       'coffeescript', 'collections', 'comments', 'crystal', 'dart', 'elixir',
       'elm (beta)', 'erlang (beta)', 'f#', 'followers', 'following',
       'fortran (beta)', 'go', 'groovy (beta)', 'haskell', 'highest trained',
       'honor', 'honor percentile', 'java', 'javascript', 'julia (beta)',
       'kotlin (beta)', 'kumite', 'last seen', 'leaderboard position',
       'lua (beta)', 'member since', 'most recent', 'name', 'nasm (beta)',
       'nim (beta)', 'objective-c (beta)', 'ocaml (beta)', 'php',
       'powershell (beta)', 'purescript (beta)', 'python', 'r (beta)', 'rank',
       'ruby', 'rust', 'scala (beta)', 'shell', 'solidity (beta)', 'sql',
       'swift', 'total completed kata', 'total languages trained',
       'translations', 'typescript', 'username', 'github', 'linkedin'],
      dtype='object')

### 4.1 Clan

No vamos a cambiar el valor de los Unknown de esta columna porque lo mismo es "" que 'Unknown'

In [93]:
# Esta columna no aporta demasiados datos debido a su baja variabilidad
# Vamos a eliminarla
df.clan.value_counts().head()
df.drop(columns=['clan'], inplace=True)

### 4.2 Collections

In [94]:
df.collections.value_counts()

# No hay que limpiar nada

0.0     10601
1.0       617
2.0        92
3.0        52
4.0        26
5.0        16
6.0        13
7.0         8
9.0         5
8.0         5
12.0        4
11.0        4
14.0        3
10.0        3
16.0        3
18.0        1
25.0        1
29.0        1
Name: collections, dtype: int64

### 4.3 Comments

In [95]:
df.comments.value_counts().head(10)

0 (0 replies)    7812
1 (0 replies)     687
1 (1 replies)     225
2 (0 replies)     222
2 (1 replies)     176
3 (0 replies)     104
3 (1 replies)      89
2 (2 replies)      65
4 (1 replies)      62
3 (2 replies)      58
Name: comments, dtype: int64

In [96]:
def clean_comments(x): 
    try: 
        res = x.split(' ')[0]
        return float(res)
    except: 
        return 0

In [97]:
# df['comments'] = df['comments'].apply(lambda x: clean_comments(x))
df['comments'] = df['comments'].apply(clean_comments)

df.comments.value_counts().head(10)

0.0     7858
1.0      980
2.0      508
3.0      311
4.0      220
5.0      164
6.0      144
7.0      107
8.0       90
10.0      78
Name: comments, dtype: int64

### 4.4 highest trained

In [98]:
# Vamos a ver si merece la pena highest trained o no. si el lenguaje coincide con su score más alto no merece la pena

# df.iloc[44]
# Podemos hacer drop a la columna

df.drop(columns=['highest trained'], inplace=True)
df.columns

Index(['allies', 'bf (beta)', 'c', 'c#', 'c++', 'clojure', 'coffeescript',
       'collections', 'comments', 'crystal', 'dart', 'elixir', 'elm (beta)',
       'erlang (beta)', 'f#', 'followers', 'following', 'fortran (beta)', 'go',
       'groovy (beta)', 'haskell', 'honor', 'honor percentile', 'java',
       'javascript', 'julia (beta)', 'kotlin (beta)', 'kumite', 'last seen',
       'leaderboard position', 'lua (beta)', 'member since', 'most recent',
       'name', 'nasm (beta)', 'nim (beta)', 'objective-c (beta)',
       'ocaml (beta)', 'php', 'powershell (beta)', 'purescript (beta)',
       'python', 'r (beta)', 'rank', 'ruby', 'rust', 'scala (beta)', 'shell',
       'solidity (beta)', 'sql', 'swift', 'total completed kata',
       'total languages trained', 'translations', 'typescript', 'username',
       'github', 'linkedin'],
      dtype='object')

### 4.5 Honor | total completed kata

In [99]:
def clean_str_float(x): 
    try: 
        return float(x.replace(',', ''))
    except: 
        return 0

In [100]:
df['honor'] = df['honor'].apply(clean_str_float)
df['honor'].value_counts().head(20)

2.0     461
4.0     400
3.0     264
6.0     167
5.0     155
7.0      94
8.0      84
0.0      78
10.0     73
12.0     65
14.0     55
13.0     52
16.0     48
46.0     45
11.0     45
42.0     43
44.0     42
9.0      38
62.0     37
48.0     37
Name: honor, dtype: int64

In [101]:
df['honor'].max()

188495.0

In [102]:
df['total completed kata'] = df['total completed kata'].apply(clean_str_float)
df['total completed kata'].value_counts().head(10)

2.0     663
1.0     580
3.0     394
4.0     284
6.0     228
5.0     216
7.0     192
9.0     187
8.0     182
15.0    182
Name: total completed kata, dtype: int64

### 4.6 honor percentile | most recent | name

* Honor percentile es un valor calculado a partir de los niveles de score de sus katas, podemos eliminarlo sin problemas. 
* most recent lo utilizaremos para nuestro modelo de entrenamiento supervisado
* name no nos interesa

In [103]:
columns_to_drop = ['honor percentile', 'name']

df.drop(columns=columns_to_drop, inplace=True)
df.columns

Index(['allies', 'bf (beta)', 'c', 'c#', 'c++', 'clojure', 'coffeescript',
       'collections', 'comments', 'crystal', 'dart', 'elixir', 'elm (beta)',
       'erlang (beta)', 'f#', 'followers', 'following', 'fortran (beta)', 'go',
       'groovy (beta)', 'haskell', 'honor', 'java', 'javascript',
       'julia (beta)', 'kotlin (beta)', 'kumite', 'last seen',
       'leaderboard position', 'lua (beta)', 'member since', 'most recent',
       'nasm (beta)', 'nim (beta)', 'objective-c (beta)', 'ocaml (beta)',
       'php', 'powershell (beta)', 'purescript (beta)', 'python', 'r (beta)',
       'rank', 'ruby', 'rust', 'scala (beta)', 'shell', 'solidity (beta)',
       'sql', 'swift', 'total completed kata', 'total languages trained',
       'translations', 'typescript', 'username', 'github', 'linkedin'],
      dtype='object')

### 4.7 kumite | translations

In [104]:
# df['kumite'] = df['kumite'].apply(lambda x: clean_honor(x))
def remove_parentesis(x): 
    try: 
        res = float(x.split(' ')[0])
        return res
    except: 
        return 0

In [105]:
df['kumite'] = df['kumite'].apply(remove_parentesis)
df['kumite'].value_counts().head(15)

0.0     9638
1.0      889
2.0      283
3.0      161
4.0      106
5.0       56
6.0       51
7.0       30
8.0       27
9.0       24
10.0      17
12.0      14
13.0      13
15.0      11
11.0      11
Name: kumite, dtype: int64

In [106]:
df['translations'] = df['translations'].apply(remove_parentesis)
df['translations'].value_counts().head(10)

0.0     10888
1.0       190
2.0       107
3.0        61
4.0        34
5.0        31
6.0        21
7.0        17
10.0        9
11.0        9
Name: translations, dtype: int64

### 4.8 last seen | member since

Convertimos estas columnas a tipo Datetime para su mejor tratamiento

In [107]:
df['member since'] = pd.to_datetime(df['member since'])
df['last seen'] = pd.to_datetime(df['last seen'])

### 4.9 Github y linkedin

Rellenamos los huecos vacios de estas columnas con strings vacios. 

In [108]:
df['linkedin'] = df['linkedin'].fillna('')
df['github'] = df['github'].fillna('')

In [109]:
df['github'].value_counts().head(), df['linkedin'].value_counts().head()

(                                    4600
 https://github.com/gayanW              1
 https://github.com/brianlaughlin       1
 https://github.com/sagynangare         1
 https://github.com/dalinaum            1
 Name: github, dtype: int64,
                                                            11015
 https://www.linkedin.com/in/maoyusu/                           1
 https://www.linkedin.com/in/ludwik-przyrowski-20a56486/        1
 https://www.linkedin.com/in/gfortaine                          1
 https://www.linkedin.com/in/timsonrisa/                        1
 Name: linkedin, dtype: int64)

Porcentaje de usuarios con github

In [112]:
len(df[df['github'] != ''])/len(df)

0.600034779584384

Porcentaje de usuarios con linkedin

In [113]:
len(df[df['linkedin'] != ''])/len(df)

0.04225719502651943

### 4.10 leaderboard position

In [114]:
def clean_leaderboard_position(x): 
    try: 
        res = float(x.replace('#', '').split(' ')[0])
        return res
    except: 
        return 0
df['leaderboard position'] = df['leaderboard position'].apply(clean_leaderboard_position)
# df['leaderboard position'].value_counts().head(10)

### 4.11 rank

In [115]:
def clean_rank(x): 
    try: 
        is_dan = 'dan' in x
        res = float(x.split(' ')[0])
        return res if not is_dan else -res
    except: 
        return 0

In [116]:
df['rank'] = df['rank'].apply(clean_rank)
df['rank'].value_counts().head(25)

 8.0    2251
 5.0    2098
 4.0    1921
 6.0    1819
 7.0    1468
 3.0    1093
 2.0     540
 1.0     242
 0.0      46
-1.0      23
Name: rank, dtype: int64

In [117]:
df.shape

(11501, 56)

In [118]:
nulos = {col: df[col].isna().sum() for col in df.columns}
nulos

{'allies': 46,
 'bf (beta)': 1,
 'c': 1,
 'c#': 1,
 'c++': 1,
 'clojure': 1,
 'coffeescript': 1,
 'collections': 46,
 'comments': 0,
 'crystal': 1,
 'dart': 1,
 'elixir': 1,
 'elm (beta)': 1,
 'erlang (beta)': 1,
 'f#': 1,
 'followers': 46,
 'following': 46,
 'fortran (beta)': 1,
 'go': 1,
 'groovy (beta)': 1,
 'haskell': 1,
 'honor': 0,
 'java': 1,
 'javascript': 1,
 'julia (beta)': 1,
 'kotlin (beta)': 1,
 'kumite': 0,
 'last seen': 71,
 'leaderboard position': 0,
 'lua (beta)': 1,
 'member since': 45,
 'most recent': 46,
 'nasm (beta)': 1,
 'nim (beta)': 1,
 'objective-c (beta)': 1,
 'ocaml (beta)': 1,
 'php': 1,
 'powershell (beta)': 1,
 'purescript (beta)': 1,
 'python': 1,
 'r (beta)': 1,
 'rank': 0,
 'ruby': 1,
 'rust': 1,
 'scala (beta)': 1,
 'shell': 1,
 'solidity (beta)': 1,
 'sql': 1,
 'swift': 1,
 'total completed kata': 0,
 'total languages trained': 46,
 'translations': 0,
 'typescript': 1,
 'username': 0,
 'github': 0,
 'linkedin': 0}

In [119]:
# Los nulos se agrupa de 46 en 46 así que voy a suponer y comprobar que son 46 registros inválidos

df.dropna(inplace=True)

nulos = {col: df[col].isna().sum() for col in df.columns}
nulos

{'allies': 0,
 'bf (beta)': 0,
 'c': 0,
 'c#': 0,
 'c++': 0,
 'clojure': 0,
 'coffeescript': 0,
 'collections': 0,
 'comments': 0,
 'crystal': 0,
 'dart': 0,
 'elixir': 0,
 'elm (beta)': 0,
 'erlang (beta)': 0,
 'f#': 0,
 'followers': 0,
 'following': 0,
 'fortran (beta)': 0,
 'go': 0,
 'groovy (beta)': 0,
 'haskell': 0,
 'honor': 0,
 'java': 0,
 'javascript': 0,
 'julia (beta)': 0,
 'kotlin (beta)': 0,
 'kumite': 0,
 'last seen': 0,
 'leaderboard position': 0,
 'lua (beta)': 0,
 'member since': 0,
 'most recent': 0,
 'nasm (beta)': 0,
 'nim (beta)': 0,
 'objective-c (beta)': 0,
 'ocaml (beta)': 0,
 'php': 0,
 'powershell (beta)': 0,
 'purescript (beta)': 0,
 'python': 0,
 'r (beta)': 0,
 'rank': 0,
 'ruby': 0,
 'rust': 0,
 'scala (beta)': 0,
 'shell': 0,
 'solidity (beta)': 0,
 'sql': 0,
 'swift': 0,
 'total completed kata': 0,
 'total languages trained': 0,
 'translations': 0,
 'typescript': 0,
 'username': 0,
 'github': 0,
 'linkedin': 0}

In [120]:
df.shape

(11428, 56)

### 4.12 Member since | Last Seen

In [121]:
# Vamos a crear una columna honor density = honor/time
# Hacer drop de las originales

In [122]:
# df.iloc[4]['member since'].year

df['days of activity'] = (df['last seen'] - df['member since']).dt.days
# df.drop(columns=['last seen', 'member since'], inplace=True)

In [123]:
df.dtypes

allies                            float64
bf (beta)                         float64
c                                 float64
c#                                float64
c++                               float64
clojure                           float64
coffeescript                      float64
collections                       float64
comments                          float64
crystal                           float64
dart                              float64
elixir                            float64
elm (beta)                        float64
erlang (beta)                     float64
f#                                float64
followers                         float64
following                         float64
fortran (beta)                    float64
go                                float64
groovy (beta)                     float64
haskell                           float64
honor                             float64
java                              float64
javascript                        

### 4.13 Guardamos los datos

In [126]:
df.to_csv('../output/df-limpio-iter3.csv')
df.shape

(11428, 57)

### Resultado

Hemos generado un CSV con registros y columnas (11500x57) que hemos limpiado. Todos salvo 4 son te tipo numérico lo que nos ayudará a análisis posteriores. 
Las columnas no numéricas son: 
* username: nos permitirá identificar al usuario
* linkedin: permite su acceso por las redes sociales y por la API propia. 
* github: permite su acceso por la API propia. 
* most recent: Permitirá ejecutar algoritmos de ML supervisado para intentar predecir este valor

In [125]:
df.head()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,honor,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,github,linkedin,days of activity
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,560.0,0.0,581.0,0.0,0.0,0.0,2018-08-01,0.0,0.0,2014-10-01,C#,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,2.0,0.0,0.0,brkiesel,,,1400
1,374.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,377.0,386.0,0.0,0.0,0.0,0.0,1984.0,14.0,4449.0,0.0,0.0,0.0,2018-09-01,0.0,0.0,2015-09-01,Python,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138.0,0.0,2.0,402.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,4.0,0.0,0.0,Em-Ant,https://github.com/Em-Ant,,1096
2,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,84.0,0.0,0.0,0.0,0.0,335.0,0.0,82.0,0.0,0.0,0.0,2017-11-01,0.0,0.0,2015-11-01,JavaScript,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,2.0,0.0,0.0,Mikeysax,https://github.com/Mikeysax,,731
3,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,30.0,0.0,0.0,0.0,0.0,3101.0,20.0,4524.0,0.0,0.0,1.0,2017-10-01,0.0,0.0,2015-07-01,Go,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5938.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.0,3.0,0.0,0.0,kimiamania,https://github.com/kimiamania,https://www.linkedin.com/in/rezhajulio,823
4,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,0.0,127.0,2.0,114.0,0.0,0.0,0.0,2018-07-01,0.0,0.0,2018-06-01,R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,2.0,0.0,0.0,Daigle,,,30
